In [ ]:
import pandas as pd
import sklearn
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFE


import lightgbm as lgb
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [ ]:
full_df = pd.read_csv("C:/Users/Kaiwei/Desktop/cs3244/Cleaned/Combined 35 Tech Companies.csv")

In [ ]:
full_df.head()

,Date,Close,Volume,OpenInt,EGS,Company_Name,Company_Number,day,year,month,...,lag42,lag43,lag44,lag45,lag46,lag47,lag48,lag49,lag50,Tomorrow_Close
0,3/1/1980,0.26459,12665490,0,2,INTC,0,3,1980,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26459
1,4/1/1980,0.26459,13431648,0,2,INTC,0,4,1980,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26459
2,7/1/1980,0.26459,11564146,0,2,INTC,0,0,1980,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.28077
3,8/1/1980,0.28077,15586456,0,2,INTC,0,1,1980,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.28868
4,9/1/1980,0.28868,19656654,0,2,INTC,0,2,1980,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.28868


In [ ]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168247 entries, 0 to 168246
Data columns (total 77 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Date            168247 non-null  object 
 1   Close           168247 non-null  float64
 2   Volume          168247 non-null  int64  
 3   OpenInt         168247 non-null  int64  
 4   EGS             168247 non-null  int64  
 5   Company_Name    168247 non-null  object 
 6   Company_Number  168247 non-null  int64  
 7   day             168247 non-null  int64  
 8   year            168247 non-null  int64  
 9   month           168247 non-null  int64  
 10  h-c             168247 non-null  float64
 11  c-l             168247 non-null  float64
 12  c-o             168247 non-null  float64
 13  ewm10           168247 non-null  float64
 14  ewm50           168247 non-null  float64
 15  ewm100          168247 non-null  float64
 16  ewm200          168247 non-null  float64
 17  ma10      

## **Regression**

https://machinelearningmastery.com/light-gradient-boosted-machine-lightgbm-ensemble/

In [ ]:
# Base Model
# df = full_df[['Volume', 'month', 'year', 'Close', 'std200', 'Tomorrow_Close']].dropna()
df = full_df.drop(['Company_Name', 'Date', 'Company_Number'], axis = 1).dropna()
# df = full_df[['Close', 'Tomorrow_Close']].dropna()
X = df.drop('Tomorrow_Close', axis = 1)
y = df['Tomorrow_Close']

print(X.shape)
print(y.shape)

(161330, 73)
(161330,)


In [ ]:
# Split into Train and Test set
lastIndex = 30000

X_train = X.head(len(X) - lastIndex)
X_test = X.tail(lastIndex)
y_train = y.head(len(X) - lastIndex)
y_test = y.tail(lastIndex)


In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
lgb_reg = lgb.LGBMRegressor()
default_params = lgb_reg.get_params()
default_params

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [ ]:
tss = TimeSeriesSplit(n_splits = 3, max_train_size = 30000)

### **LightGBM Boosting**

In [ ]:
# Base LightGBM Boost
from sklearn.metrics import mean_squared_error

params = {
    'objective': 'regression',
    'linear_tree': True,
    'metric':'root_mean_squared_error'
}

lgb_reg = lgb.LGBMRegressor(**params, random_state=0)

lgb_reg.fit(X_train, y_train)

predicted_ytest = lgb_reg.predict(X_test)
predicted_ytrain = lgb_reg.predict(X_train)
print("Train Score: " + str(np.sqrt(mean_squared_error(y_train.to_numpy(), predicted_ytrain))))
print("Test Score: " + str(np.sqrt(mean_squared_error(y_test.to_numpy(), predicted_ytest))))

cv_results = cross_val_score(lgb_reg, X_train, y_train, cv = tss, scoring = 'neg_root_mean_squared_error')

print("Mean RMSE: %f" % (cv_results.mean()))

Train Score:0.7744285106996938
Test Score:15.991604577348422
Mean RMSE: -15.995301


In [ ]:
# LightGBM Boost
from sklearn.metrics import mean_squared_error

params = {
    'objective': 'regression',
    'linear_tree': True,
    'metric':'root_mean_squared_error',
    'learning_rate': 0.05, 
    'max_depth':3, 'num_leaves':18, 'reg_alpha':0.5,'reg_lambda':0
}

param_grid = { 
    # reg_alpha provides l1 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [0, 0.5],
    # reg_lambda provides l2 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [0, 0.5],
    # apply a weighting factor for the corrections by new trees 
    "learning_rate": [0.05, 0.1, 0.2],
    # how big tree should be
    "max_depth": [3, 5, 7, 9, 11],
    # tune number of decision trees
    "n_estimators": [10, 50, 100, 500],
    # large num_leaves helps improve accuracy but might lead to over-fitting
    "num_leaves": [6, 10, 14, 18, 22],
    "feature_fraction":[0.1,0.5,0.7,1]
}

params = {
    'objective': 'regression',
    'linear_tree': True,
    'metric':'root_mean_squared_error'
}

lgb_reg = lgb.LGBMRegressor(**params, random_state=0)

cv = HalvingGridSearchCV(lgb_reg, param_grid, verbose = 0, cv = tss, n_jobs = -1,scoring='neg_root_mean_squared_error', random_state=0)
model = cv.fit(X_train, y_train)
print(model.best_estimator_)

# cv_results = cross_val_score(model, X_train, y_train, cv = tss, scoring = 'neg_root_mean_squared_error')

print("Mean RMSE: %f" % (model.best_score_))

# model.fit(X_train, y_train) 
predicted_ytest = model.predict(X_test)
predicted_ytrain = model.predict(X_train)
print("Train Score:" + str(np.sqrt(mean_squared_error(y_train.to_numpy(), predicted_ytrain))))
print("Test Score:" + str(np.sqrt(mean_squared_error(y_test.to_numpy(), predicted_ytest))))

### **XGBoost**

In [ ]:
# XGBoost on LR
from xgboost import XGBRegressor

# Use HalvingGridSeach CV to find the best validation score and hyperparameters

param_grid = { 
    # reg_alpha provides l1 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [0, 0.5],
    # reg_lambda provides l2 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [0, 0.5],
    # apply a weighting factor for the corrections by new trees 
    "learning_rate": [0.05, 0.1, 0.2],
    # tune number of decision trees
    "n_estimators": [10, 50, 100, 500]
}

model = XGBRegressor(booster='gblinear', eval_metric='rmse',random_state=0)

cv = HalvingGridSearchCV(model, param_grid, verbose = 0, cv = tss, n_jobs = -1, scoring='neg_root_mean_squared_error')
model = cv.fit(X_train, y_train)
print(model.best_estimator_)

#model.fit(X_train, y_train)#, eval_metric='root_mean_squared_error')
#cv_results = cross_val_score(model, X, y, cv = tss, scoring = 'neg_root_mean_squared_error')

print("Mean RMSE: %f" % (model.best_score_))

predicted_ytest = model.predict(X_test)
predicted_ytrain = model.predict(X_train)
print("Train Score:" + str(np.sqrt(mean_squared_error(y_train.to_numpy(), predicted_ytrain))))
print("Test Score:" + str(np.sqrt(mean_squared_error(y_test.to_numpy(), predicted_ytest))))

XGBRegressor(base_score=0.5, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, gpu_id=-1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=0, num_parallel_tree=None, predictor=None,
             random_state=0, ...)
Mean RMSE: -1.675583
Train Score:1.3346583219982735
Test Score:3.510480323775975


In [ ]:
# XGBoost on DT
from xgboost import XGBRegressor

# Use HalvingGridSeach CV to find the best validation score and hyperparameters

param_grid = { 
    # reg_alpha provides l1 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [0, 0.5],
    # reg_lambda provides l2 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [0, 0.5],
    # apply a weighting factor for the corrections by new trees 
    "learning_rate": [0.05, 0.1, 0.2],
    # tune number of decision trees
    "n_estimators": [10, 50, 100, 500]
}

model = XGBRegressor(eval_metric='rmse',random_state=0)

cv = HalvingGridSearchCV(model, param_grid, verbose = 0, cv = tss, n_jobs = -1, scoring='neg_root_mean_squared_error')
model = cv.fit(X_train, y_train)
print(model.best_estimator_)

print("Mean RMSE: %f" % (model.best_score_))

predicted_ytest = model.predict(X_test)
predicted_ytrain = model.predict(X_train)
print("Train Score:" + str(np.sqrt(mean_squared_error(y_train.to_numpy(), predicted_ytrain))))
print("Test Score:" + str(np.sqrt(mean_squared_error(y_test.to_numpy(), predicted_ytest))))

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)
Mean RMSE: -11.237747
Train Score:0.7627409302387541
Test Score:43.887098098939994


### **Default Linear Regression**

> Let us compare our scores with Baseline Models

In [ ]:
# Default Linear Regression 
from sklearn.linear_model import LinearRegression

model = LinearRegression()

cv_results = cross_val_score(model, X_train, y_train, cv = tss, scoring = 'neg_root_mean_squared_error')

print("Validation RMSE: %f" % (-1 * cv_results.mean()))

model.fit(X_train, y_train)
predicted_ytest = model.predict(X_test)
predicted_ytrain = model.predict(X_train)
print("Train Score:" + str(np.sqrt(mean_squared_error(y_train.to_numpy(), predicted_ytrain))))
print("Test Score:" + str(np.sqrt(mean_squared_error(y_test.to_numpy(), predicted_ytest))))

Validation RMSE: 1.263493
Train Score:1.0601852609352977
Test Score:2.731424739360334


In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()

cv_results = cross_val_score(model, X_train, y_train, cv = tss, scoring = 'neg_root_mean_squared_error')

print("Mean RMSE: %f" % (cv_results.mean()))

model.fit(X_train, y_train) # eval_metric='neg_mean_absolute_error', eval_set=[(X_test, y_test)], verbose=0)
predicted_ytest = model.predict(X_test)
predicted_ytrain = model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train.to_numpy(), predicted_ytrain)))
print(np.sqrt(mean_squared_error(y_test.to_numpy(), predicted_ytest)))

Mean RMSE: -11.434768
3.610275080065969e-17
43.76605903770178


## **Classification**

In [ ]:
# Create a new target variable, 'increase' 
increase = []
numVariables = full_df.shape[0]

for i in range(numVariables):
    if full_df['Tomorrow_Close'][i] > full_df['Close'][i]:
        increase.append(1)
    else:
        increase.append(0)

In [ ]:
full_df['increase'] = increase
full_df.head()

,Date,Close,Volume,OpenInt,EGS,Company_Name,Company_Number,day,year,month,...,lag43,lag44,lag45,lag46,lag47,lag48,lag49,lag50,Tomorrow_Close,increase
0,3/1/1980,0.26459,12665490,0,2,INTC,0,3,1980,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26459,0
1,4/1/1980,0.26459,13431648,0,2,INTC,0,4,1980,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26459,0
2,7/1/1980,0.26459,11564146,0,2,INTC,0,0,1980,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.28077,1
3,8/1/1980,0.28077,15586456,0,2,INTC,0,1,1980,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.28868,1
4,9/1/1980,0.28868,19656654,0,2,INTC,0,2,1980,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.28868,0


In [ ]:
# Base Model
df = full_df.drop(['Company_Name', 'Date', 'Company_Number', 'Tomorrow_Close'], axis = 1).dropna()
X = df.drop('increase', axis = 1)
y = df['increase']

print(df['increase'].sum()/len(df['increase']))
print(X.shape)
print(y.shape)

0.49098121862021943
(161330, 73)
(161330,)


In [ ]:
# Split into Train and Test set
lastIndex = 30000

X_train = X.head(len(X) - lastIndex)
X_test = X.tail(lastIndex)
y_train = y.head(len(X) - lastIndex)
y_test = y.tail(lastIndex)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
lgb_class = lgb.LGBMClassifier()

cv_results = cross_val_score(lgb_class, X_train, y_train, cv = tss, scoring = 'accuracy')

print("Mean Accuracy: %f" % (cv_results.mean()))

Mean Accuracy: 0.508528


### **LightGBM**

In [ ]:
# LightGBM Boost
param_grid = { 
    # reg_alpha provides l1 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [0, 0.5],
    # reg_lambda provides l2 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [0, 0.5],
    # apply a weighting factor for the corrections by new trees 
    "learning_rate": [0.05, 0.3, 0.1],
    # how big tree should be
    "max_depth": [3, 7, 11],
    # tune number of decision trees
    "n_estimators": [10, 100, 500],
    # large num_leaves helps improve accuracy but might lead to over-fitting
    "num_leaves": [6, 18, 22],
    "feature_fraction": [1]
}


lgb_reg = lgb.LGBMClassifier(random_state=0)

cv = HalvingGridSearchCV(lgb_reg, param_grid, verbose = 0, cv = tss, n_jobs = -1,scoring='accuracy',random_state=0)
model = cv.fit(X_train, y_train)

print("Mean accuracy: %f" % (model.best_score_))


predicted_ytest = model.predict(X_test)
predicted_ytrain = model.predict(X_train)
print("Train Score:" + str(accuracy_score(y_train.to_numpy(), predicted_ytrain)))
print("Test Score:" + str(accuracy_score(y_test.to_numpy(), predicted_ytest)))
print("Confusion Score:" + str(confusion_matrix(y_test.to_numpy(), predicted_ytest)))

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
Mean accuracy: 0.512072
Train Score:0.5698164928043858
Test Score:0.5217333333333334
Confusion Score:[[ 3599 10366]
 [ 3982 12053]]


### **XGBoost**

In [ ]:
# XGBoost on dt

from xgboost import XGBClassifier

# Use HalvingGridSeach CV to find the best validation score and hyperparameters

param_grid = { 
    # reg_alpha provides l1 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [0, 0.5],
    # reg_lambda provides l2 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [0, 0.5],
    # apply a weighting factor for the corrections by new trees 
    "learning_rate": [0.05, 0.1, 0.2],
    # tune number of decision trees
    "n_estimators": [10, 50, 100, 500]
}

model = XGBClassifier(random_state=0)

cv = HalvingGridSearchCV(model, param_grid, verbose = 0, cv = tss, n_jobs = -1, scoring='accuracy',random_state=0)
model = cv.fit(X_train, y_train)

print("Mean accuracy: %f" % (model.best_score_))

predicted_ytest = model.predict(X_test)
predicted_ytrain = model.predict(X_train)
print("Train Score:" + str(accuracy_score(y_train.to_numpy(), predicted_ytrain)))
print("Test Score:" + str(accuracy_score(y_test.to_numpy(), predicted_ytest)))
print("Confusion Score:" + str(confusion_matrix(y_test.to_numpy(), predicted_ytest)))

Mean accuracy: 0.509813
Train Score:0.5863321404096551
Test Score:0.5241
Confusion Score:[[ 3771 10194]
 [ 4083 11952]]


### **Default Decision Tree**

> Once again, let us compare with a baseline model


In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=0)

cv_results = cross_val_score(model, X_train, y_train, cv = tss, scoring = 'accuracy')

print("Mean accuracy: %f" % (cv_results.mean()))

model.fit(X_train, y_train)
predicted_ytest = model.predict(X_test)
predicted_ytrain = model.predict(X_train)
print("Train Score:" + str(accuracy_score(y_train.to_numpy(), predicted_ytrain)))
print("Test Score:" + str(accuracy_score(y_test.to_numpy(), predicted_ytest)))
print("Confusion Score:" + str(confusion_matrix(y_test.to_numpy(), predicted_ytest)))

Mean accuracy: 0.501360
Train Score:1.0
Test Score:0.5012666666666666
Confusion Score:[[5968 7997]
 [6965 9070]]
